In [ ]:
# Install a pip package in the current Jupyter kernel - https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install librosa matplotlib
sys.path.append('.')

Code from the article [Comparative Audio Analysis With Wavenet, MFCCs, UMAP, t-SNE and PCA](https://medium.com/@LeonFedden/comparative-audio-analysis-with-wavenet-mfccs-umap-t-sne-and-pca-cb8237bfce2f).

In [5]:
import os
import librosa
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
# directory = '/Users/bjornpjo/Downloads/test-sounds/'
directory = '/Users/bjornpjo/Documents/MCT/MCT4052-79d7e16aaa8bdbfafa6205bd1f4acebb3ba59b44/notebooks/workshops/data/examples2/'

# sample_rate = 44100
sample_rate = 48000
mfcc_size = 13

for file in os.listdir(directory):
    print(file)
    if file.endswith('.wav'):
        file_path = os.path.join(directory, file)
        pcm_data, _ = librosa.load(file_path)
        print(pcm_data)
        # Compute a vector of n * 13 mfccs
        mfccs = librosa.feature.mfcc(y=pcm_data, 
                             sr=sample_rate, 
                             n_mfcc=mfcc_size)
        # print mfccs dimensions
        print("shape:", mfccs.shape)
        print(mfccs)

snare_28.wav
[-5.6695938e-04  2.5057495e-03  3.1134903e-02 ...  2.3531215e-04
  6.1143655e-05  2.1575484e-05]
shape: (13, 6)
[[ 6.51274185e+01  7.01214066e+01  1.00042276e+01 -5.58898239e+01
  -1.35169205e+02 -2.41192764e+02]
 [ 5.99452591e+01  5.40507240e+01  3.74385757e+01  3.61216125e+01
   4.18689499e+01  5.05172310e+01]
 [-1.66381586e+00  2.72521043e+00  1.09014435e+01  1.05857868e+01
   1.38545170e+01  1.15689363e+01]
 [-9.65952492e+00 -6.28284872e-01  1.85069008e+01  2.32825317e+01
   2.90621643e+01  2.77575703e+01]
 [-3.38144684e+01 -3.04033699e+01 -2.49080791e+01 -2.30367661e+01
  -2.07349319e+01 -1.59217663e+01]
 [ 2.00603104e+00 -2.33995646e-01 -3.62906885e+00 -1.99588442e+00
   1.51986194e+00  4.14154959e+00]
 [-8.52751541e+00 -1.13499069e+01 -1.51136560e+01 -1.08041954e+01
  -8.62726974e+00 -8.39802170e+00]
 [-6.32963371e+00 -4.24481869e+00  3.27638865e-01  1.65124977e+00
   4.33284521e+00  5.04603243e+00]
 [-1.53907118e+01 -1.28469181e+01 -9.31464005e+00 -4.02132511e+00
 

In [3]:
# Get the standard deviation
stddev_features = np.std(mfccs, axis=1)
print("stddev_features:", stddev_features.shape)

# Get the mean
mean_features = np.mean(mfccs, axis=1)

# Get the average difference of the features
average_difference_features = np.zeros((mfcc_size,))
for i in range(0, len(mfccs.T) - 2, 2):
    average_difference_features += mfccs.T[i] - mfccs.T[i+1]
average_difference_features /= (len(mfccs.T) // 2)   
average_difference_features = np.array(average_difference_features)

# Concatenate the features to a single feature vector
concat_features_features = np.hstack((stddev_features, mean_features))
concat_features_features = np.hstack((concat_features_features, average_difference_features))

print("concat_features_features:", concat_features_features, concat_features_features.shape)

stddev_features: (13,)
concat_features_features: [ 1.15094559e+02  2.99994450e+01  1.33553114e+01  2.63211517e+01
  1.45737238e+01  1.47188034e+01  1.06117249e+01  9.88883972e+00
  7.65306711e+00  6.23634672e+00  5.11313200e+00  4.54049540e+00
  4.34068203e+00 -5.10634125e+02  1.49040995e+01  6.87893867e+00
  1.29468708e+01  6.98122692e+00  7.64042616e+00  5.47853184e+00
  5.36632109e+00  3.67772055e+00  2.62092137e+00  2.12786078e+00
  1.44637787e+00  1.64119971e+00  9.59488632e+00  1.77930207e+00
  6.56803232e-01  1.35585974e+00 -3.59127807e-01 -7.62970227e-02
 -1.53289049e-01 -1.23472652e-02 -7.19918311e-02 -6.23208705e-02
 -3.07010168e-02 -3.25705562e-01 -2.46263520e-01] (39,)


In [9]:
def get_pca(features):
    pca = PCA(n_components=2)
    transformed = pca.fit(features).transform(features)
    scaler = MinMaxScaler()
    scaler.fit(transformed)
    return scaler.transform(transformed)
  

# my_array_of_feature_vectors = ...
concat_features_features = np.array(concat_features_features)  # [:, None]
print("concat_features_features.shape:", concat_features_features.shape)
scaled_pca = get_pca(concat_features_features)
print(scaled_pca)


# plt.scatter(scaled_pca[:, 0], scaled_pca[:, 1])
# plt.show()


concat_features_features.shape: (39,)


ValueError: Expected 2D array, got 1D array instead:
array=[ 1.15094559e+02  2.99994450e+01  1.33553114e+01  2.63211517e+01
  1.45737238e+01  1.47188034e+01  1.06117249e+01  9.88883972e+00
  7.65306711e+00  6.23634672e+00  5.11313200e+00  4.54049540e+00
  4.34068203e+00 -5.10634125e+02  1.49040995e+01  6.87893867e+00
  1.29468708e+01  6.98122692e+00  7.64042616e+00  5.47853184e+00
  5.36632109e+00  3.67772055e+00  2.62092137e+00  2.12786078e+00
  1.44637787e+00  1.64119971e+00  9.59488632e+00  1.77930207e+00
  6.56803232e-01  1.35585974e+00 -3.59127807e-01 -7.62970227e-02
 -1.53289049e-01 -1.23472652e-02 -7.19918311e-02 -6.23208705e-02
 -3.07010168e-02 -3.25705562e-01 -2.46263520e-01].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.